# Creamos la estructura de datos para la opción 3

In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
import keras 
import os
import json

import random as python_random
import time
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.metrics import MeanAbsolutePercentageError
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.models import load_model

from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor

from keras import layers, models

np.random.seed(123)
python_random.seed(123)
tf.random.set_seed(1234)

from keras_tuner import HyperModel, HyperParameters
from keras_tuner.tuners import Hyperband
from tensorflow.keras.layers import Conv1D, BatchNormalization, MaxPooling1D

from funcionesComunes import *

2024-12-10 12:32:37.047126: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-10 12:32:37.168283: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-10 12:32:37.283993: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-10 12:32:37.389318: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-10 12:32:37.420192: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-10 12:32:37.738482: I tensorflow/core/platform/cpu_feature_gu

## Hiperparametros de TCNN

In [4]:
class TCNNModel(HyperModel):

    def __init__(self, input_shape, num_tcnn_layers_input=3, num_tcnn_layers_after_input=3):
        self.input_shape = input_shape # (SEQ_LENGTH, num_features)
        self.num_tcnn_layers_input = num_tcnn_layers_input
        self.num_tcnn_layers_after_input = num_tcnn_layers_after_input

        # Para controlar las dimensiones temporales
        self.temporalDimension = self.input_shape[0]

    def build(self, hp):
        
        # Podemos ver como "hp.Int" permite ir modificando los parámetros del modelo con INT
        # Con "hp.Float" lo hace pero para valores decimales 
        # IMPORTANTE!!! Tenemos que controlar las dimensionalidad temporal teniendo en cuenta la reducción que se hace en kernel y Pooling
        
        model = Sequential()
        
        # Numero de capas de TCNN
        num_tcnn_layers_input = hp.Int("num_tcnn_layers_input", min_value = 1, 
                                       max_value = self.num_tcnn_layers_input, default = self.num_tcnn_layers_input)

        for i in range(num_tcnn_layers_input):

            # Añadiendo capas TCNN
            # Solo la primera capa necesita input_shape

            if i == 0:
                    
                # Primera capa de convolución con padding 'same'
                model.add(layers.Conv1D(filters=hp.Int(f'units_tcnn_{i}', min_value=32, max_value=128, step=32),
                                        kernel_size=3, activation='relu', 
                                        padding='same', input_shape=self.input_shape)) # Dimensión temporal despues de la convolución: 4 (3+1) (USAMOS PADDING 'SAME')
                model.add(layers.BatchNormalization())
                model.add(layers.MaxPooling1D(pool_size=2)) # Reducción de la dimensión temporal a la mitad: 4/2=2 (REDONDEA HACIA ABAJO)

                # Obtenemos el redondeo hacia abajo
                self.temporalDimension = math.floor(self.temporalDimension / 2)
    
            else:

                # Segunda capa de convolución con padding 'same'
                model.add(layers.Conv1D(filters=hp.Int(f'units_tcnn_{i}', min_value=32, max_value=128, step=32),
                                        kernel_size=3, activation='relu', 
                                        padding='same')) # Dimensión temporal despues de la convolución: 2 (USAMOS PADDING 'SAME')
                model.add(layers.BatchNormalization())

                if self.temporalDimension >= 2:
                    model.add(layers.MaxPooling1D(pool_size=1)) # Reducción de la dimensión temporal a la mitad: 2/2=1 (REDONDEA HACIA ABAJO)
                    self.temporalDimension = math.floor(self.temporalDimension / 2)
                else:
                    model.add(layers.MaxPooling1D(pool_size=1)) # Reducción de la dimensión temporal a la mitad: 2/1=2 (REDONDEA HACIA ABAJO)

        # Se modifica la capa Dropout desde 0.0 hasta 0.5 con un step de 0.05
        model.add(Dropout(hp.Float('dropout', min_value=0.0, max_value=0.5, default=0.25, step=0.05)))
        
        # Creamos un bucle que permite incrementar las capas TCNN destras de la capa Dropout
        num_tcnn_layers_after_input = hp.Int("num_tcnn_layers_after_input", min_value = 1, 
                                       max_value = self.num_tcnn_layers_after_input, default = self.num_tcnn_layers_after_input)

        for i in range(num_tcnn_layers_input):

            # Segunda capa de convolución con padding 'same'
            model.add(layers.Conv1D(filters=hp.Int(f'units_tcnn_{i}', min_value=32, max_value=128, step=32),
                                    kernel_size=3, activation='relu',
                                    padding='same')) # USAMOS PADDING 'SAME'
            model.add(layers.BatchNormalization())

            if self.temporalDimension >= 2:
                model.add(layers.MaxPooling1D(pool_size=2)) # REDONDEA HACIA ABAJO
                self.temporalDimension = math.floor(self.temporalDimension / 2)
            else:
                model.add(layers.MaxPooling1D(pool_size=1)) # REDONDEA HACIA ABAJO

        # Capa final de convolución con padding 'same'
        model.add(layers.Conv1D(filters=hp.Int(f'units_tcnn_{i}', min_value=32, max_value=128, step=32),
                                kernel_size=3, activation='relu', padding='same')) # Dimensión temporal despues de la convolución: 1 (USAMOS PADDING 'SAME')	
        model.add(layers.BatchNormalization())
        model.add(layers.GlobalAveragePooling1D()) # Promedio global de la dimensión temporal
    
        # Capa densa
        model.add(layers.Dense(hp.Int('dense_units', min_value=4, max_value=64, step=4), 
                               activation=hp.Choice('dense_activation',values=['relu', 'sigmoid', 'tanh', 'elu', 'relu'])))
        
        # Capa de salida
        model.add(layers.Dense(1, activation=hp.Choice('dense_activation',values=['relu', 'sigmoid', 'tanh', 'elu', 'relu'],default='relu')))
        
        # Variaciones de los optimizadores
        optimizer = hp.Choice('optimizer', ['adam', 'sgd', 'rmsprop'])
        
        if optimizer == 'adam':
            opt = Adam(
                learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')
            )
        elif optimizer == 'sgd':
            opt = SGD(
                learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')
            )
        else: # rmsprop
            opt = RMSprop(
                learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')
            )
        
        
        model.compile(optimizer=opt,
                        loss=MeanSquaredError(),
                        metrics=[RootMeanSquaredError()])
            
        return model

    def fit(self, hp, model, *args, **kwargs):
        return model.fit(*args,
            batch_size=hp.Choice("batch_size", [16, 24, 32]),
            **kwargs)
        

Ajuste de hiperparametros

In [ ]:
# Cargar datos
data_train_global = np.load(f'RCPMergedTransferTotal/totalMerged_train_transfer.npz', allow_pickle=True)
data_val_global = np.load(f'RCPMergedTransferTotal/totalMerged_val_transfer.npz', allow_pickle=True)
data_val = np.load("RCPMergedTransferTotal/totalMerged_val.npz", allow_pickle=True)

X_train_transfer = data_train_global['X_train_transfer']
y_train_transfer = data_train_global['y_train_transfer']
X_val_transfer = data_val_global['X_val_transfer']
y_val_transfer = data_val_global['y_val_transfer']
X_val = data_val['X_val']
y_val = data_val['y_val']

valorNormalizacion = data_train_global["valorNormalizacion"].item()

WINDOWS_SIZE = 3

# Crear modelo de hiperparámetros
hypermodel = TCNNModel(input_shape=(WINDOWS_SIZE + 1, X_train_transfer.shape[2]), num_tcnn_layers_input=3, num_tcnn_layers_after_input=3)

tuner = Hyperband(
            hypermodel,
            objective='val_loss',
            max_epochs=50,
            factor=3,
            directory='best_models_26_11_24/TCNN_Option3',
            project_name=f'hyperparameter_tuning_totalMerged'
        )

tuner.search(X_train_transfer, y_train_transfer, epochs=50, validation_data=(X_val_transfer, y_val_transfer))

top_models_data = []

best_trials = tuner.oracle.get_best_trials(num_trials=10)

for i, trial in enumerate(best_trials):
    hyperparameters = trial.hyperparameters.values
    model = tuner.hypermodel.build(trial.hyperparameters)
    model.load_weights(tuner.get_trial_dir(trial.trial_id) + "/checkpoint.weights.h5")
    
    # Guardamos el modelo
    model.save(f"models/TCNNMerged_Option3/totalMerged_model_{i+1}.keras")

    # Entrenamiento
    predictions_train = predictionForIndividuals(X_train_transfer, y_train_transfer, model, hyperparameters["batch_size"])
    predictions_train["PredictionsDenormalize"] = predictions_train.apply(lambda row: desnormalizacionBAI(row, valorNormalizacion, "Predictions"), axis=1)
    predictions_train["ActualDenormalize"] = predictions_train.apply(lambda row: desnormalizacionBAI(row, valorNormalizacion, "Actuals"), axis=1)

    train_mse = mean_squared_error(predictions_train["ActualDenormalize"], predictions_train["PredictionsDenormalize"])
    train_rmse = np.sqrt(train_mse)
    train_mape = mean_absolute_percentage_error(predictions_train["ActualDenormalize"], predictions_train["PredictionsDenormalize"]) *100
    train_r2 = r2_score(predictions_train["ActualDenormalize"], predictions_train["PredictionsDenormalize"])

    # Validación
    predictions_val = predictionForIndividuals(X_val_transfer, y_val_transfer, model, hyperparameters["batch_size"])
    predictions_val["PredictionsDenormalize"] = predictions_val.apply(lambda row: desnormalizacionBAI(row, valorNormalizacion, "Predictions"), axis=1)
    predictions_val["ActualDenormalize"] = predictions_val.apply(lambda row: desnormalizacionBAI(row, valorNormalizacion, "Actuals"), axis=1)

    val_mse = mean_squared_error(predictions_val["ActualDenormalize"], predictions_val["PredictionsDenormalize"])
    val_rmse = np.sqrt(val_mse)
    val_mape = mean_absolute_percentage_error(predictions_val["ActualDenormalize"], predictions_val["PredictionsDenormalize"]) *100
    val_r2 = r2_score(predictions_val["ActualDenormalize"], predictions_val["PredictionsDenormalize"])

    # Prueba
    predictions_test = predictionForIndividuals(X_val, y_val, model, hyperparameters["batch_size"])
    predictions_test["PredictionsDenormalize"] = predictions_test.apply(lambda row: desnormalizacionBAI(row, valorNormalizacion, "Predictions"), axis=1)
    predictions_test["ActualDenormalize"] = predictions_test.apply(lambda row: desnormalizacionBAI(row, valorNormalizacion, "Actuals"), axis=1)

    test_mse = mean_squared_error(predictions_test["ActualDenormalize"], predictions_test["PredictionsDenormalize"])
    test_rmse = np.sqrt(test_mse)
    test_mape = mean_absolute_percentage_error(predictions_test["ActualDenormalize"], predictions_test["PredictionsDenormalize"]) *100
    test_r2 = r2_score(predictions_test["ActualDenormalize"], predictions_test["PredictionsDenormalize"])

    # Almacenar la información del modelo
    model_info = {
        'Model Rank': i+1,
        'Trial ID': trial.trial_id,
        # Métricas del conjunto de entrenamiento
        'train_mse': train_mse,
        'train_rmse': train_rmse,
        'train_mape': train_mape,
        'train_r2': train_r2,
        # Métricas del conjunto de validación
        'val_mse': val_mse,
        'val_rmse': val_rmse,
        'val_mape': val_mape,
        'val_r2': val_r2,
        # Métricas del conjunto de prueba
        'test_mse': test_mse,
        'test_rmse': test_rmse,
        'test_mape': test_mape,
        'test_r2': test_r2
    }

    print(f"RESULTADOS DE MSE, RMSE, R2, MAPE (Train): {train_mse}, {train_rmse}, {train_r2}, {train_mape}")
    print(f"RESULTADOS DE MSE, RMSE, R2, MAPE (Val): {val_mse}, {val_rmse}, {val_r2}, {val_mape}")
    print(f"RESULTADOS DE MSE, RMSE, R2, MAPE (Test): {test_mse}, {test_rmse}, {test_r2}, {test_mape}")

    # Agregamos los hyperparametros
    model_info.update(hyperparameters)
    top_models_data.append(model_info)

with open(f"models/TCNNMerged_Option3/totalMerged_best_models.json", 'w') as file:
    json.dump(top_models_data, file, indent=4)

Reloading Tuner from best_models_26_11_24/TCNN_Option3/hyperparameter_tuning_totalMerged/tuner0.json


/home/workstation/Escritorio/PabloC/TransferLearning/funcionesComunes.py:247: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  predictionsDF = pd.concat([predictionsDF, tempDF], ignore_index=True)
/home/workstation/Escritorio/PabloC/TransferLearning/funcionesComunes.py:247: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  predictionsDF = pd.concat([predictionsDF, tempDF], ignore_index=True)
/home/workstation/Escritorio/PabloC/TransferLearning/funcionesComunes.py:247: FutureWarning: The behavior of Dat

RESULTADOS DE MSE, RMSE, R2, MAPE (Train): 37.69391932443064, 6.139537386841996, 0.8447621429766812, 30.714720688274582
RESULTADOS DE MSE, RMSE, R2, MAPE (Val): 41.742366455395484, 6.460833263240546, 0.8460422259436728, 30.657899404459314
RESULTADOS DE MSE, RMSE, R2, MAPE (Test): 39.8437506796274, 6.312190640310811, 0.8395951685382154, 30.4647682060158


/home/workstation/Escritorio/PabloC/TransferLearning/.venv/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/home/workstation/Escritorio/PabloC/TransferLearning/.venv/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 50 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
/home/workstation/Escritorio/PabloC/TransferLearning/funcionesComunes.py:247: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. T

RESULTADOS DE MSE, RMSE, R2, MAPE (Train): 40.149226175566454, 6.336341702872916, 0.8346502580696119, 32.608209972473375
RESULTADOS DE MSE, RMSE, R2, MAPE (Val): 41.47446311760228, 6.440067011887554, 0.84703032999841, 32.41297925703394
RESULTADOS DE MSE, RMSE, R2, MAPE (Test): 41.26124638924332, 6.423491759879771, 0.8338885481392728, 31.929180218425813


...

RESULTADOS DE MSE, RMSE, R2, MAPE (Train): 48.39446503269742, 6.9566130431911635, 0.8006932370495978, 44.895937856865345
RESULTADOS DE MSE, RMSE, R2, MAPE (Val): 48.56472630221678, 6.968839666846755, 0.8208794135537656, 43.19317599902323
RESULTADOS DE MSE, RMSE, R2, MAPE (Test): 50.283225849620926, 7.091066622844615, 0.7975674420659555, 42.19310751862318


/home/workstation/Escritorio/PabloC/TransferLearning/.venv/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/home/workstation/Escritorio/PabloC/TransferLearning/.venv/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 50 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
/home/workstation/Escritorio/PabloC/TransferLearning/funcionesComunes.py:247: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. T

RESULTADOS DE MSE, RMSE, R2, MAPE (Train): 43.96312573519754, 6.630469495834933, 0.8189431730768456, 30.231333029592832
RESULTADOS DE MSE, RMSE, R2, MAPE (Val): 45.590205270409385, 6.752051930369714, 0.8318502969949368, 29.99487648332308
RESULTADOS DE MSE, RMSE, R2, MAPE (Test): 45.36252320974017, 6.73517061474616, 0.8173774364605649, 30.112046897045836


/home/workstation/Escritorio/PabloC/TransferLearning/.venv/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/home/workstation/Escritorio/PabloC/TransferLearning/.venv/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 34 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
/home/workstation/Escritorio/PabloC/TransferLearning/funcionesComunes.py:247: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. T

RESULTADOS DE MSE, RMSE, R2, MAPE (Train): 44.36569404375674, 6.6607577679838155, 0.8172852440886619, 34.971865707958905
RESULTADOS DE MSE, RMSE, R2, MAPE (Val): 46.17355686741609, 6.795112719257576, 0.8296987296307978, 33.711627447583275
RESULTADOS DE MSE, RMSE, R2, MAPE (Test): 45.71197362372402, 6.761063054263288, 0.8159706026709943, 33.172010347866795


# Aplicamos transfer

In [2]:
# Creamos un df que contiene los resultados de los modelos freeze de cada especie
df_Freeze = pd.DataFrame(columns = ["File", "Time", "TrainMSE", "TrainRMSE", "TrainR2", "TrainMAPE", 
                                    "ValidationMSE", "ValidationRMSE", "ValidationR2", "ValidationMAPE",
                                    "TestMSE", "TestRMSE", "TestR2", "TestMAPE"])

for archivo in os.listdir("RCPMergedTransfer"):
    if "train" in archivo: 
        
        nombreArchivo = archivo.split("_train")[0]
        print(f"PROCESANDO ARCHIVO: {nombreArchivo}")

        # Cargar datos
        data_transfer_train = np.load(f'RCPMergedTransferTotal/totalMerged_train_transfer.npz', allow_pickle=True)
        data_transfer_val = np.load(f'RCPMergedTransfer/{nombreArchivo}_val.npz', allow_pickle=True)
        data_transfer_test = np.load(f'RCPMergedTransfer/{nombreArchivo}_test.npz', allow_pickle=True)
        
        # Aqui tenemos que usar TRAIN original para aplicar transfer 
        X_train_transfer = data_transfer_train["X_train_transfer"]
        y_train_transfer = data_transfer_train["y_train_transfer"]

        # Aqui tenemos que usar VAL original para aplicar transfer 
        X_val_transfer = data_transfer_val["X_val"]
        y_val_transfer = data_transfer_val["y_val"]
        
        # Este test es para ver la capacidad del modelo
        X_test_transfer = data_transfer_test["X_test"]
        y_test_transfer = data_transfer_test["y_test"]

        # Valor normalizacion
        valorNormalizacion_train = data_transfer_train["valorNormalizacion"].item()
        valorNormalizacion = data_transfer_test["valorNormalizacion"].item()

        # Cargamos el conjunto de datos de val original para hacer el transfer
        model = tf.keras.models.load_model('models/TCNNMerged_Option3/totalMerged_model_1.keras')
        modelFreeze = model
        
        # Obtenemos el valor de batch size
        with open(f'models/TCNNMerged_Option3/totalMerged_best_models.json') as f:
            parameters = json.load(f)

        # Obtener el optimizador del modelo
        optimizer = model.optimizer

        batch_size_LSTM = parameters[0]["batch_size"]

        # Indicamos el numero de layers a entrenar
        NUM_TRAINABLE = 1

        numTCNN_layers = sum(1 for layer in model.layers if isinstance(layer, (Conv1D)))

        numFreezeLayers = numTCNN_layers - NUM_TRAINABLE # Congelamos todas menos la útlima capa

        numberLayersFreezed = 0

        # Recorre las capas en orden inverso
        for i, layer in enumerate(model.layers):

            if isinstance(layer, (Conv1D)):
                numberLayersFreezed +=1
        
            if isinstance(layer, (Conv1D, BatchNormalization, MaxPooling1D)) and numFreezeLayers>=numberLayersFreezed:
                
                layer.trainable = False
            
            print(layer, layer.trainable)


        # Compilamos el modelo con los nuevos datos
        model.compile(optimizer=optimizer, loss=MeanSquaredError(), metrics=[
                                    MeanSquaredError(name='mse'),
                                    RootMeanSquaredError(name='rmse'),
                                    MeanAbsolutePercentageError(name='mape')
                                ])

        # Comienza a medir el tiempo de entrenamiento
        start_time = time.time()

        historyLSTMTransfer = modelFreeze.fit(X_train_transfer, y_train_transfer, epochs=200, batch_size=batch_size_LSTM,
                                validation_data=(X_val_transfer, y_val_transfer), callbacks=[EarlyStopping(monitor='val_loss', patience=10)])
        
        # Finaliza la medición del tiempo de entrenamiento
        end_time = time.time()
        print(f"[MODELO CONGELADO]: {end_time-start_time}")

        # Guardamos el modelo
        modelFreeze.save(f"models/TCNNMerged_Option3/{nombreArchivo}.keras")

        # Realizar predicciones y calcular métricas para el conjunto de entrenamiento
        predictions_train = predictionForIndividuals(X_train_transfer, y_train_transfer, modelFreeze, batch_size_LSTM)
        predictions_train["PredictionsDenormalize"] = predictions_train.apply(lambda row: desnormalizacionBAI(row, valorNormalizacion_train, "Predictions"), axis=1)
        predictions_train["ActualDenormalize"] = predictions_train.apply(lambda row: desnormalizacionBAI(row, valorNormalizacion_train, "Actuals"), axis=1)

        train_mse = mean_squared_error(predictions_train["ActualDenormalize"],predictions_train["PredictionsDenormalize"])
        train_rmse = np.sqrt(train_mse)
        train_mape = (np.sum(np.abs(predictions_train["PredictionsDenormalize"] - predictions_train["ActualDenormalize"])) / np.sum(np.abs(predictions_train["ActualDenormalize"]))) * 100
        train_r2 = r2_score(predictions_train["ActualDenormalize"], predictions_train["PredictionsDenormalize"])

        # Realizar predicciones y calcular métricas para el conjunto de validación
        predictions_val = predictionForIndividuals(X_val_transfer, y_val_transfer, modelFreeze, batch_size_LSTM)
        predictions_val["PredictionsDenormalize"] = predictions_val.apply(lambda row: desnormalizacionBAI(row, valorNormalizacion, "Predictions"), axis=1)
        predictions_val["ActualDenormalize"] = predictions_val.apply(lambda row: desnormalizacionBAI(row, valorNormalizacion, "Actuals"), axis=1)

        val_mse = mean_squared_error(predictions_val["ActualDenormalize"],predictions_val["PredictionsDenormalize"])
        val_rmse = np.sqrt(val_mse)
        val_mape = (np.sum(np.abs(predictions_val["PredictionsDenormalize"] - predictions_val["ActualDenormalize"])) / np.sum(np.abs(predictions_val["ActualDenormalize"]))) * 100
        val_r2 = r2_score(predictions_val["ActualDenormalize"], predictions_val["PredictionsDenormalize"])

        # Realizar predicciones y calcular métricas para el conjunto de prueba
        predictions_test = predictionForIndividuals(X_test_transfer, y_test_transfer, modelFreeze, batch_size_LSTM)
        predictions_test["PredictionsDenormalize"] = predictions_test.apply(lambda row: desnormalizacionBAI(row, valorNormalizacion, "Predictions"), axis=1)
        predictions_test["ActualDenormalize"] = predictions_test.apply(lambda row: desnormalizacionBAI(row, valorNormalizacion, "Actuals"), axis=1)

        test_mse = mean_squared_error(predictions_test["ActualDenormalize"],predictions_test["PredictionsDenormalize"])
        test_rmse = np.sqrt(test_mse)
        test_mape = (np.sum(np.abs(predictions_test["PredictionsDenormalize"] - predictions_test["ActualDenormalize"])) / np.sum(np.abs(predictions_test["ActualDenormalize"]))) * 100
        test_r2 = r2_score(predictions_test["ActualDenormalize"], predictions_test["PredictionsDenormalize"])

        print(f"RESULTADOS DE MSE, RMSE, R2, MAPE (Train): {train_mse}, {train_rmse}, {train_r2}, {train_mape}")
        print(f"RESULTADOS DE MSE, RMSE, R2, MAPE (Val): {val_mse}, {val_rmse}, {val_r2}, {val_mape}")
        print(f"RESULTADOS DE MSE, RMSE, R2, MAPE (Test): {test_mse}, {test_rmse}, {test_r2}, {test_mape}")

        # Guardamos los datos calculados
        df_Freeze.loc[len(df_Freeze)] = [nombreArchivo, end_time-start_time,train_mse, train_rmse, train_r2, train_mape, val_mse, val_rmse, val_r2, val_mape, test_mse, test_rmse, test_r2, test_mape]

df_Freeze.to_csv("resultsTransferOption3/resultados_Option3_TCNN.csv", index=False)

PROCESANDO ARCHIVO: Juniperus spp. L


I0000 00:00:1733830406.612721   17657 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1733830406.613009   17657 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-12-10 12:33:26.649376: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


<Conv1D name=conv1d_12, built=True> False
<BatchNormalization name=batch_normalization_12, built=True> False
<MaxPooling1D name=max_pooling1d_10, built=True> False
<Conv1D name=conv1d_13, built=True> False
<BatchNormalization name=batch_normalization_13, built=True> False
<MaxPooling1D name=max_pooling1d_11, built=True> False
<Conv1D name=conv1d_14, built=True> False
<BatchNormalization name=batch_normalization_14, built=True> False
<MaxPooling1D name=max_pooling1d_12, built=True> False
<Dropout name=dropout_2, built=True> True
<Conv1D name=conv1d_15, built=True> False
<BatchNormalization name=batch_normalization_15, built=True> False
<MaxPooling1D name=max_pooling1d_13, built=True> False
<Conv1D name=conv1d_16, built=True> False
<BatchNormalization name=batch_normalization_16, built=True> False
<MaxPooling1D name=max_pooling1d_14, built=True> False
<Conv1D name=conv1d_17, built=True> False
<BatchNormalization name=batch_normalization_17, built=True> False
<MaxPooling1D name=max_poolin

/home/workstation/Escritorio/PabloC/TransferLearning/.venv/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 66 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


4016/4016 ━━━━━━━━━━━━━━━━━━━━ 14s 3ms/step - loss: 0.0149 - mape: 31.6930 - mse: 0.0149 - rmse: 0.1221 - val_loss: 0.0194 - val_mape: 36.4047 - val_mse: 0.0194 - val_rmse: 0.1392
Epoch 2/200
4016/4016 ━━━━━━━━━━━━━━━━━━━━ 13s 3ms/step - loss: 0.0144 - mape: 30.7729 - mse: 0.0144 - rmse: 0.1202 - val_loss: 0.0195 - val_mape: 36.2386 - val_mse: 0.0195 - val_rmse: 0.1395
Epoch 3/200
4016/4016 ━━━━━━━━━━━━━━━━━━━━ 13s 3ms/step - loss: 0.0143 - mape: 30.3789 - mse: 0.0143 - rmse: 0.1195 - val_loss: 0.0191 - val_mape: 36.0787 - val_mse: 0.0191 - val_rmse: 0.1383
Epoch 4/200
4016/4016 ━━━━━━━━━━━━━━━━━━━━ 13s 3ms/step - loss: 0.0143 - mape: 30.0854 - mse: 0.0143 - rmse: 0.1195 - val_loss: 0.0190 - val_mape: 35.7350 - val_mse: 0.0191 - val_rmse: 0.1380
Epoch 5/200
4016/4016 ━━━━━━━━━━━━━━━━━━━━ 13s 3ms/step - loss: 0.0142 - mape: 30.0603 - mse: 0.0142 - rmse: 0.1193 - val_loss: 0.0190 - val_mape: 35.8714 - val_mse: 0.0190 - val_rmse: 0.1379
Epoch 6/200
4016/4016 ━━━━━━━━━━━━━━━━━━━━ 13s 3ms/s

...

RESULTADOS DE MSE, RMSE, R2, MAPE (Train): 35.48721764580363, 5.957114875995227, 0.8538501774878017, 21.425420997519083
RESULTADOS DE MSE, RMSE, R2, MAPE (Val): 8.677728377706119, 2.945798427880991, 0.9060939306707038, 23.836310640216634
RESULTADOS DE MSE, RMSE, R2, MAPE (Test): 10.438852614066427, 3.230921325886229, 0.8841059065943168, 24.936977168281075
PROCESANDO ARCHIVO: Pinus wallichiana
<Conv1D name=conv1d_12, built=True> False
<BatchNormalization name=batch_normalization_12, built=True> False
<MaxPooling1D name=max_pooling1d_10, built=True> False
<Conv1D name=conv1d_13, built=True> False
<BatchNormalization name=batch_normalization_13, built=True> False
<MaxPooling1D name=max_pooling1d_11, built=True> False
<Conv1D name=conv1d_14, built=True> False
<BatchNormalization name=batch_normalization_14, built=True> False
<MaxPooling1D name=max_pooling1d_12, built=True> False
<Dropout name=dropout_2, built=True> True
<Conv1D name=conv1d_15, built=True> False
<BatchNormalization name=bat

/home/workstation/Escritorio/PabloC/TransferLearning/.venv/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 66 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


4016/4016 ━━━━━━━━━━━━━━━━━━━━ 12s 3ms/step - loss: 0.0149 - mape: 31.6547 - mse: 0.0149 - rmse: 0.1220 - val_loss: 0.0196 - val_mape: 37.6500 - val_mse: 0.0196 - val_rmse: 0.1399
Epoch 2/200
4016/4016 ━━━━━━━━━━━━━━━━━━━━ 12s 3ms/step - loss: 0.0144 - mape: 30.5563 - mse: 0.0144 - rmse: 0.1201 - val_loss: 0.0195 - val_mape: 38.2938 - val_mse: 0.0195 - val_rmse: 0.1396
Epoch 3/200
4016/4016 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - loss: 0.0143 - mape: 30.3647 - mse: 0.0143 - rmse: 0.1197 - val_loss: 0.0193 - val_mape: 38.2730 - val_mse: 0.0193 - val_rmse: 0.1388
Epoch 4/200
4016/4016 ━━━━━━━━━━━━━━━━━━━━ 12s 3ms/step - loss: 0.0142 - mape: 30.1177 - mse: 0.0142 - rmse: 0.1194 - val_loss: 0.0194 - val_mape: 38.1011 - val_mse: 0.0195 - val_rmse: 0.1395
Epoch 5/200
4016/4016 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - loss: 0.0142 - mape: 30.1268 - mse: 0.0142 - rmse: 0.1191 - val_loss: 0.0193 - val_mape: 37.9216 - val_mse: 0.0193 - val_rmse: 0.1388
Epoch 6/200
4016/4016 ━━━━━━━━━━━━━━━━━━━━ 12s 3ms/s

/home/workstation/Escritorio/PabloC/TransferLearning/funcionesComunes.py:247: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  predictionsDF = pd.concat([predictionsDF, tempDF], ignore_index=True)
/home/workstation/Escritorio/PabloC/TransferLearning/funcionesComunes.py:247: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  predictionsDF = pd.concat([predictionsDF, tempDF], ignore_index=True)
/home/workstation/Escritorio/PabloC/TransferLearning/funcionesComunes.py:247: FutureWarning: The behavior of Dat

RESULTADOS DE MSE, RMSE, R2, MAPE (Train): 35.57715277242924, 5.964658646765064, 0.8534797905241029, 21.59416303322401
RESULTADOS DE MSE, RMSE, R2, MAPE (Val): 8.111307744656694, 2.8480357695535874, 0.9076936961421492, 23.95243140266023
RESULTADOS DE MSE, RMSE, R2, MAPE (Test): 10.34033121743052, 3.2156385396108376, 0.8908624825166929, 24.698819911659754
